In [14]:
#!pip install -r requirements.txt
import os
import pandas as pd
import openai
from openai import OpenAI
import pickle
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import ast 
import re
import tensorflow as tf
api_key_ccl = ''
api_key_indv = ''
api_key = api_key_indv

print('All modules are ready.')

All modules are ready.


## Call API to do Embedding

In [7]:
client = OpenAI(api_key=api_key)
# embedding model parameters
embedding_model = "text-embedding-ada-002"

# input data folder path
input_folder = "./data/five_years"
# output data folder path
output_folder = "./data/five_years/five_years_with_embeddings"
# create output folder if not exists
os.makedirs(output_folder, exist_ok=True)
# get all csv files in the input folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith("_label.csv")]

for csv_file in csv_files:
    # construct input file path
    input_datapath = os.path.join(input_folder, csv_file)

    # load & inspect dataset
    df = pd.read_csv(input_datapath, index_col=False)

    # remove empty rows
    df = df.dropna()

    # create a list to store the embeddings
    responses = []
    for question in df["QUESTION_cut"]:
        response = client.embeddings.create(
            input=question,
            model=embedding_model
        ).data[0].embedding
        response = np.asarray(response, dtype=float)
        responses.append(response)
        print(f"{question} Embedding: {response}")

    # add the embeddings to the DataFrame
    df["Q_embedding"] = responses

    # construct output file path
    output_datapath = os.path.join(output_folder, csv_file.replace("_label.csv", "_label_embedding.pkl"))

    # Save the DataFrame to a pickle file
    with open(output_datapath, "wb") as f:
        pickle.dump(df, f)

print("操作完成！")

問卷 類別 Embedding: [-8.42111837e-03 -1.57833360e-02  8.46941315e-04 ...  7.47334052e-05
 -2.12924797e-02 -5.79175493e-03]
訪問 時段 Embedding: [ 0.00850868 -0.01405382  0.00997276 ... -0.00146143  0.01769635
  0.01348799]
現 居 地址 Embedding: [-0.01151653  0.00546734 -0.00755062 ... -0.01163691 -0.00215015
 -0.02499929]
受 訪 者 與 戶籍地 戶長 之 關係 Embedding: [ 0.01375262 -0.00964603  0.02067692 ... -0.0019676  -0.01509634
 -0.00735621]
受 訪 者 性別 Embedding: [-0.0075576  -0.01707484  0.02253591 ... -0.01603738 -0.00465776
 -0.0004379 ]
__年 出生 Embedding: [-0.01019418 -0.01442931 -0.01172806 ...  0.02051053 -0.00247389
 -0.0174835 ]
出生 月 __ 月 Embedding: [-0.01158345 -0.02710684 -0.00239188 ...  0.01845209  0.00435693
 -0.02613499]
第一 次 接受 我們 訪問 嗎 Embedding: [ 0.01758088 -0.02116217 -0.00553811 ... -0.01760801 -0.01132719
  0.02269508]
出生地 Embedding: [ 0.01452097 -0.0292876  -0.01996634 ...  0.0052543  -0.02160404
 -0.01031071]
A4 地區 碼 Embedding: [-0.015102    0.01329571 -0.00412967 ... -0.00729599 -0.034340

In [8]:
#type_of_type_part_1

# 新增每年小檔案中的type
import os
import pandas as pd

# 输入数据文件夹路径
input_folder1 = "./data/five_years/five_years_with_embeddings"
input_folder2 = "./data/five_years/"

# 获取指定路径下所有以"_label_embedding.pkl"结尾的文件
pkl_files1 = [f for f in os.listdir(input_folder1) if f.endswith("_label_embedding.pkl")]
pkl_files2 = [f for f in os.listdir(input_folder2) if f.endswith("_label.csv")]

for pkl_file in pkl_files1:
    # 构建输入文件的完整路径
    input_filepath = os.path.join(input_folder1, pkl_file)

    # 读取DataFrame
    df = pd.read_pickle(input_filepath)

    # 建立 "type" 列
    df['type'] = df['NUMBER'].str[0]

    # 将更新后的数据框存回原始.pkl文件
    df.to_pickle(input_filepath)

for pkl_file in pkl_files2:
    # 构建输入文件的完整路径
    input_filepath = os.path.join(input_folder2, pkl_file)

    # 读取DataFrame
    df = pd.read_csv(input_filepath)

    # 建立 "type" 列
    df['type'] = df['NUMBER'].str[0]

    # 将更新后的数据框存回原始.csv文件
    df.to_csv(input_filepath, index=False)

print("操作完成！")

操作完成！


In [11]:
input_folder = "./data/five_years/five_years_with_embeddings"
pkl = [f for f in os.listdir(input_folder) if f.endswith("_label_embedding.pkl")]
for file in pkl:
    path = os.path.join(input_folder, file)
    df = pd.read_pickle(path)
    print(df)


        YEAR NUMBER               QUESTION  ANSWER               QUESTION_cut  \
0    CII2002    x10              受訪者與戶長之關係    X10F            受 訪 者 與 戶長 之 關係   
1    CII2002    a01                     性別    A01F                         性別   
2    CII2002  a03z1              請問您的出生地是:  A03Z1F                        出生地   
3    CII2002  a03z2  若出生地是台灣:____縣市，_____鄉    LABA  若 出生地 台灣 :____ 縣市 _____ 鄉   
4    CII2002   a04a       請問您認為自己目前的健康狀況如_    LABB         認為 自己 目前 健康 狀況 如 _   
..       ...    ...                    ...     ...                        ...   
267  CII2002   g05b         家俱與家庭耐久設備____元   LABAK       家俱 與 家庭 耐久 設備 ____ 元   
268  CII2002   g05c              衣著費用____元   LABAK               衣著 費用 ____ 元   
269  CII2002   g05d              教育費用____元   LABAK               教育 費用 ____ 元   
270  CII2002   g05e              醫療費用____元   LABAK               醫療 費用 ____ 元   
271  CII2002   g05f           紅白帖費用____元 ;   LABAK            紅白帖 費用 ____ 元     

    type                   

## **Data Preprocessing**將相似的大題人工分成同一類

In [12]:
#type_of_type_part_2

#手動的similarity_list
similarity_list = [[['CVIII2014', 'x'], ['CIII2004', 'X'], ['CV2008', 'x'], ['CII2002', 'x'], ['CX2018', 'x']],
                   [['CVIII2014', 'a'], ['CIII2004', 'A'], ['CV2008', 'a'], ['CII2002', 'a'], ['CX2018', 'a']],
                   [['CVIII2014', 'b'], ['CIII2004', 'B'], ['CV2008', 'b'], ['CII2002', 'b'], ['CX2018', 'b']],
                   [['CVIII2014', 'c'], ['CIII2004', 'C'], ['CV2008', 'c'], ['CII2002', 'c'], ['CX2018', 'c']],
                   [['CVIII2014', 'd'], ['CIII2004', 'D'], ['CV2008', 'd'], ['CII2002', 'd'], ['CX2018', 'd']],
                   [['CVIII2014', 'e'],['CIII2004', 'F'],['CV2008', 'e'], ['CII2002', 'f'],  ['CX2018', 'e']],
                   [['CVIII2014', 'f'],['CIII2004', 'G'], ['CV2008', 'f'], ['CII2002', 'g'],['CX2018', 'f']],
                   [['CIII2004', 'E'], ['CII2002', 'e']],
                   [['CX2018', 'g']]]

import os
import pandas as pd
import pickle

# 定義讀入檔案的路徑
base_path = "./data/five_years/five_years_with_embeddings/"

# 定義輸出檔案的路徑
output_base_path = "./data/five_years/five_years_by_type_v2/"

# 確認輸出路徑存在
os.makedirs(output_base_path, exist_ok=True)

# 迭代處理每個大類
for i, sublist in enumerate(similarity_list):
    type_name = f"type_{i+1}"
    print(type_name)
    # 收集該大類下的所有行
    all_rows = pd.DataFrame()  # 儲存所有行的DataFrame
    for item in sublist:
        folder_name, label = item
        # 讀取對應的pkl檔案
        pkl_file = os.path.join(base_path, f"{folder_name}_label_embedding.pkl")
        
        #print(pkl_file)
        if os.path.exists(pkl_file):  # 檢查檔案是否存在
            print(pkl_file)
            df = pd.read_pickle(pkl_file)

            # 篩選符合條件的行
            filtered_rows = df[df['type'] == label]

            # 將篩選後的資料與之前的資料合併
            #print('before', all_rows)
            all_rows = pd.concat([all_rows, filtered_rows])
            
            # 重置索引为默认整数索引
            all_rows.reset_index(drop=True, inplace=True)
            #print('after', all_rows)
            
        else:
            print(f"檔案不存在: {pkl_file}")

    # 重置索引为默认整数索引
    all_rows.reset_index(drop=True, inplace=True)

    # 儲存all_rows到目標pkl檔案
    target_pkl_file = os.path.join(output_base_path, f"{type_name}.pkl")
    with open(target_pkl_file, 'wb') as f:
        pickle.dump(all_rows, f)
        
print('操作完成！')

type_1
./data/five_years/five_years_with_embeddings/CVIII2014_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CIII2004_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CV2008_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CII2002_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CX2018_label_embedding.pkl
type_2
./data/five_years/five_years_with_embeddings/CVIII2014_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CIII2004_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CV2008_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CII2002_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CX2018_label_embedding.pkl
type_3
./data/five_years/five_years_with_embeddings/CVIII2014_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CIII2004_label_embedding.pkl
./data/five_years/five_years_with_embeddings/CV2008_label_embedding.pkl
./data/five_years/five_yea

## **Survey Questions Clustering** Using GPT4o-mini


In [15]:
# 設定 API 金鑰
openai.api_key = api_key
# 定義從YEAR列提取年份的函数
def extract_year(year_string):
    match = re.search(r'\d{4}', year_string)
    return int(match.group()) if match else None

In [17]:
# 指定文件夹路径
folder_path = './data/five_years/five_years_by_type_v2/'

# 修改保存文件夹路径
output_folder_path = './data/five_years/five_years_by_type_v2/'

# 获取文件夹中的.pkl文件列表
pkl_files = [file for file in os.listdir(folder_path) if file.startswith('type_7') and file.endswith('.pkl')]

for pkl_file in pkl_files:
    # 构建.pkl文件的完整路径
    pkl_path = os.path.join(folder_path, pkl_file)
    # 加载.pkl文件
    print(f'Now reading {pkl_path}')
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
        
    # 新建一个名为'cluster' 'similarity'的列
    data['cluster'] = None
    data['similarity'] = None
                                                                                                    
    
    # 将'cluster'分配给'YEAR'为'CV2008'的数据
    refer_indices = data[data['YEAR'] == 'CV2008'].index
    data.loc[refer_indices, 'cluster'] = range(1, len(refer_indices)+1)

    # 提取 CV2008 的数据，作为 GPT 提示的参考
    refer_data = data.loc[refer_indices][['QUESTION', 'cluster']]

    # 创建一个字符串，作为参考数据发送到 GPT
    reference_text = "\n".join([f"Question: {row['QUESTION']}, Cluster: {row['cluster']}" for _, row in refer_data.iterrows()])
    print(reference_text)
    # 分类 'cluster' 为 None 的数据
    none_indices = data[data['cluster'].isna()].index

    # 使用 GPT-4o-mini 進行分類
    for index in none_indices:
        print(f"Question data:{data.iloc[index]['QUESTION']}")
        # 準備發送到 GPT-4o-mini 的提示內容
        prompt = f"""
        Based on the previous clustering data, here is the reference:
        {reference_text}
        Now, based on the following data, please:
        1. Assign a cluster number.
        2. Provide the similarity between 0 and 1.
        Format the response as: Cluster: X, Similarity: Y
        Data: {data.iloc[index]['QUESTION']}

        If the data does not fit into any existing clusters then format the response as: Cluster: 0, Similarity: 0
        """

        try:
            # 使用 GPT-4o-mini API 進行分類
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an expert in clustering survey data."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=20,  # 限制回應的長度以只包含 cluster 編號
                temperature=0.1  # 設定較低的隨機性以提高準確性
            )

            # 從 GPT 的回應中獲取預測的 Cluster 值
            generated_text = response.choices[0].message.content.strip()
            print(f"Generated cluster for index {index}: {generated_text}")

            # 使用正则表达式提取cluster编号和相似度
            cluster_match = re.search(r'Cluster:\s*(\d+)', generated_text)
            similarity_match = re.search(r'Similarity:\s*([0-9]*\.?[0-9]+)', generated_text)
            predicted_cluster = cluster_match.group(1) if cluster_match else 0
            predicted_similarity = similarity_match.group(1) if similarity_match else 0


            # 将预测的 Cluster 值和相似度设置给对应的数据
            data.at[index, 'cluster'] = predicted_cluster
            data.at[index, 'similarity'] = predicted_similarity
            print(index,"  ;cluster:", predicted_cluster, "  ;similarity:", predicted_similarity)
        except Exception as e:
            print(f"Error while processing index {index}: {str(e)}")
    
    
    ###### 对于每个YEAR，处理相同cluster的情况
    # 将'cluster'和'similarity'列转换为数值类型
    data['cluster'] = pd.to_numeric(data['cluster'], errors='coerce')
    data['similarity'] = pd.to_numeric(data['similarity'], errors='coerce')
    for year in data['YEAR'].unique():
        year_data = data[data['YEAR'] == year]
        # 获取该YEAR中所有非0的cluster
        clusters = year_data['cluster'].dropna().unique()
        clusters = clusters[clusters != 0]
        for cluster in clusters:
            cluster_data = year_data[year_data['cluster'] == cluster]
            if len(cluster_data) > 1:
                # 找到具有最高similarity的记录
                max_similarity = cluster_data['similarity'].max()
                # 获取具有最高similarity的记录的索引
                max_similarity_indices = cluster_data[cluster_data['similarity'] == max_similarity].index
                # 获取其他需要设置cluster为0的记录的索引
                other_indices = cluster_data[~cluster_data.index.isin(max_similarity_indices)].index
                # 将其他记录的cluster设置为0
                data.loc[other_indices, 'cluster'] = 0
                data.loc[other_indices, 'similarity'] = 0

    ##### 將難以分類(cluster=0)的資料再進行一次分類
    data['YEAR_EXTRACTED'] = data['YEAR'].apply(extract_year)
    # Step 1: 处理 cluster = 0 的数据
    cluster_0_data = data[data['cluster'] == 0]
    median_year = cluster_0_data['YEAR_EXTRACTED'].median()
    min_year = cluster_0_data.loc[cluster_0_data['YEAR_EXTRACTED'] < median_year, 'YEAR_EXTRACTED'].max()
    selected_data = cluster_0_data[cluster_0_data['YEAR_EXTRACTED'] == min_year]

    # 若沒有符合條件年份的資料:
    if selected_data.empty:
        print("没有找到符合条件的数据。")
    original_max_cluster = data['cluster'].max()
    
    # 给median年份且cluster為零的資料按顺序贴标
    for i, index in enumerate(selected_data.index):
        data.at[index, 'cluster'] = data['cluster'].max() +1

    print('第二次分類總共有', max(data['cluster']), '個cluster')
    # Step 2: 使用 GPT-4o-mini 进行分类
    # 创建参考数据用于 GPT 分类
    refer_data = data[data['cluster'] > original_max_cluster][['QUESTION', 'cluster']]
    reference_text_2 = "\n".join([f"Question: {row['QUESTION']}, Cluster: {row['cluster']}" for _, row in refer_data.iterrows()])
    print(f"Reference: {reference_text_2}")
    # 对 cluster = 0 的数据进行 GPT 分类
    none_indices = data[data['cluster'] == 0].index
    for index in none_indices:
        print(f"Question data: {data.iloc[index]['QUESTION']}")
        prompt = f"""
        Based on the previous clustering data, here is the reference:
        {reference_text_2}
        Now, based on the following data, please:
        1. Assign a cluster number.
        2. Provide the similarity between 0 and 1.
        Format the response as: Cluster: X, Similarity: Y
        Data: {data.iloc[index]['QUESTION']}

        If the data does not fit into any existing clusters then format the response as: Cluster: 0, Similarity: 0
        """
        try:
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an expert in clustering survey data."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=20,
                temperature=0.1
            )

            generated_text = response.choices[0].message.content.strip()
            cluster_match = re.search(r'Cluster:\s*(\d+)', generated_text)
            similarity_match = re.search(r'Similarity:\s*([0-9]*\.?[0-9]+)', generated_text)
            predicted_cluster = int(cluster_match.group(1)) if cluster_match else 0
            predicted_similarity = float(similarity_match.group(1)) if similarity_match else 0

            # 更新数据
            data.at[index, 'cluster'] = predicted_cluster
            data.at[index, 'similarity'] = predicted_similarity
            print(index,"  ;cluster:", predicted_cluster, "  ;similarity:", predicted_similarity)
        except Exception as e:
            print(f"Error processing index {index}: {e}")

    # Step 3: 对相同年份但相似度较低的 cluster 进行处理
    for year in data['YEAR_EXTRACTED'].unique():
        year_data = data[data['YEAR_EXTRACTED'] == year]
        clusters = year_data['cluster'].unique()
        clusters = clusters[clusters != 0]
        
        for cluster in clusters:
            cluster_data = year_data[year_data['cluster'] == cluster]
            if len(cluster_data) > 1:
                max_similarity = cluster_data['similarity'].max()
                max_similarity_indices = cluster_data[cluster_data['similarity'] == max_similarity].index
                other_indices = cluster_data[~cluster_data.index.isin(max_similarity_indices)].index
            
                # 将相似度较低的设置为 cluster 0
                data.loc[other_indices, 'cluster'] = 0
                data.loc[other_indices, 'similarity'] = 0
                
    # 保存更新後的.pkl文件
    updated_pkl_path = os.path.join(output_folder_path, pkl_file)
    with open(updated_pkl_path, 'wb') as f:
        pickle.dump(data, f)

    # 構建輸出的新.csv文件路徑
    csv_file_name = pkl_file.replace('.pkl', '.csv')
    csv_path = os.path.join(output_folder_path, csv_file_name)

    # 構建包含指定列的新數據框
    output_data = data[['ANSWER', 'NUMBER', 'QUESTION', 'YEAR', 'cluster', 'similarity']]

    # 保存新的.csv文件
    output_data.to_csv(csv_path, index=False)

    # 打印輸出路徑
    print(f"聚類結果已保存至 {csv_path} 和 {updated_pkl_path}")



Now reading ./data/five_years/five_years_by_type_v2/type_7.pkl
Question: 在去年(96年)之中，請問您平均每週大約花多少時間作家務工作?__小時, Cluster: 1
Question: 請問您的配偶平均每週大約花多少時間作家務工作?__ 小時, Cluster: 2
Question: 在去年(96年)之中，您家庭自政府得到的補助總計大約是多少?__ 元, Cluster: 3
Question: 在過去一年裡，您(和您配偶)每個月的平均支出房屋貸款支出平均每月 __元?, Cluster: 4
Question: 標會支出活會平均每月 __元, Cluster: 5
Question: 死會平均每月 __元, Cluster: 6
Question: 保姆或幫傭(包括家事管理)支出平均每月 __元, Cluster: 7
Question: 請問您(和您配偶)過去一年裡，人壽或商業醫療保險 __元, Cluster: 8
Question: 請問您(和您配偶)過去一年裡，家俱與家庭耐久設備 __元, Cluster: 9
Question: 請問您(和您配偶)過去一年裡，教育費用 __元, Cluster: 10
Question: 訪問結束時間:月, Cluster: 11
Question: 訪問結束時間:日, Cluster: 12
Question: 訪問結束時間:時, Cluster: 13
Question: 訪問結束時間:分 ;, Cluster: 14
Question data:在去年(102年)之中，請問您平均每週大約花多少時間作家務工作?__小時
Generated cluster for index 0: Cluster: 1, Similarity: 1.0
0   ;cluster: 1   ;similarity: 1.0
Question data:在去年(102年)之中，請問您的配偶平均每週大約花多少時間作家務工作?__小時
Generated cluster for index 1: Cluster: 2, Similarity: 1
1   ;cluster: 2   ;similarity: 1
Question data:在去年之中，您的家庭是否曾

查看輸出結果

In [18]:

pd.set_option('display.max_rows', 200) 
output_data

,ANSWER,NUMBER,QUESTION,YEAR,cluster,similarity
0,F01A,f01a,在去年(102年)之中，請問您平均每週大約花多少時間作家務工作?__小時,CVIII2014,1,1.00
1,F01B,f01b,在去年(102年)之中，請問您的配偶平均每週大約花多少時間作家務工作?__小時,CVIII2014,2,1.00
2,F02A01F,f02a01,在去年之中，您的家庭是否曾經得到政府的中低收入戶生活補助?,CVIII2014,15,0.00
3,F02A02F,f02a02,在去年之中，您的家庭是否曾經得到政府的傷病醫療費用補助?,CVIII2014,16,0.00
4,F02A03F,f02a03,在去年之中，您的家庭是否曾經得到政府的教育補助(含五歲幼兒學費補助)?,CVIII2014,17,0.00
5,F02A04F,f02a04,在去年之中，您的家庭是否曾經得到政府的兒童托育補助?,CVIII2014,18,0.00
6,F02A05F,f02a05,在去年之中，您的家庭是否曾經得到政府的老人津貼(含老農津貼)?,CVIII2014,19,0.00
7,F02A06F,f02a06,在去年之中，您的家庭是否曾經得到政府的榮民就養金?,CVIII2014,20,0.00
8,F02A07F,f02a07,在去年之中，您的家庭是否曾經得到政府的身心障礙者補助?,CVIII2014,21,0.00
9,F02A08F,f02a08,在去年之中，您的家庭是否曾經得到政府的重大傷病補助?,CVIII2014,22,0.00


## **Answer Organizing**

In [19]:
# 加上答案做第一個整併的大表_part1(製作variable 對應到題目相同的不同年份)
import os
import csv
import pickle

# 输入和输出目录
input_directory = './data/five_years/five_years_by_type_v2/'
output_directory = './data/five_years/five_years_by_type_v2/'

# 创建输出目录
os.makedirs(output_directory, exist_ok=True)

# 处理每个小檔案
for filename in os.listdir(input_directory):
    if filename.endswith(".csv") and filename.startswith("type_7"):  # 仅处理CSV文件
        # 读取CSV文件
        csv_file_path = os.path.join(input_directory, filename)
        with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
            reader = csv.DictReader(csv_file)
            data = [row for row in reader]

        # 创建新的CSV文件并写入数据
        output_filename = f'variable_map_{os.path.splitext(filename)[0]}.csv'
        output_file_path = os.path.join(output_directory, output_filename)

        with open(output_file_path, 'w', encoding='utf-8', newline='') as file:
            fieldnames = ['ANSWER_new', 'OPTION_dict', 'OPTION_new', 'OPTION_pre', 'ANSWER_pre', 'YEAR', 'NUMBER']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()

            processed_clusters = set()  # 用于存储已处理的cluster

            for row in data:
                cluster = row['cluster']

                if cluster in processed_clusters:
                    continue  # 如果已处理过该cluster，则跳过该行数据

                processed_clusters.add(cluster)

                year = [row['YEAR']]
                number = [row['NUMBER']]
                answer_pre = [row['ANSWER']]

                # 查找属于同一cluster的行，并将相关数据添加到option_pre和answer_pre中
                for other_row in data:
                    if other_row['cluster'] == cluster and other_row != row:
                        answer_pre.append(other_row['ANSWER'])
                        number.append(other_row['NUMBER'])
                        year.append(other_row['YEAR'])

                answer_new = cluster

                writer.writerow({
                    'ANSWER_new': answer_new,
                    'OPTION_dict':'',
                    'OPTION_new': '',
                    'OPTION_pre': '',
                    'ANSWER_pre': ', '.join(answer_pre),
                    'YEAR': ', '.join(year),
                    'NUMBER': ', '.join(number)
                })

        print(f'{output_filename} 已创建。')

variable_map_type_7.csv 已创建。


In [23]:
#加上答案做第一個整併的大表_part2
import os
import csv
import re

# 输入和输出目录
input_directory = './data/five_years/five_years_by_type_v2/'
data_folder = './data/five_years'

def keep_chinese(input_string):
    # 使用正则表达式只匹配中文字符和中文标点符号
    chinese_pattern = re.compile(r'[\u4e00-\u9fa5，。！？]')
    result = re.findall(chinese_pattern, input_string)
    # 将匹配到的字符列表连接成一个字符串
    filtered_string = ''.join(result)
    return filtered_string

# 处理每个小檔案
for filename in os.listdir(input_directory):
    if filename.startswith("variable_map_") and filename.endswith("7.csv"):  # 仅处理answer_cluster文件
        # 读取CSV文件
        csv_file_path = os.path.join(input_directory, filename)
        with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
            reader = csv.DictReader(csv_file)
            data = [row for row in reader]

        # 获取cluster
        cluster = filename.split('_')[2].split('.')[0]

        # 填充OPTION_new字段，并确保选项唯一且不带数字
        for row in data:
            year_list = row['YEAR'].split(', ')
            number_list = row['ANSWER_pre'].split(', ')
            option_pre = []
            option_new = []
            option = []

            for year, number in zip(year_list, number_list):
                answer_file = os.path.join(data_folder, year + '_answer.csv')

                if os.path.isfile(answer_file):
                    with open(answer_file, 'r', encoding='utf-8') as answer_csv:
                        option_dict = dict()
                        reader = csv.DictReader(answer_csv)
                        for answer_row in reader:
                            if answer_row['ANSWER'] == number:
                                option_text = answer_row['OPTION']
                                option_pre.append(option_text)
                                option_text_new = keep_chinese(option_text)  # 清理选项中的数字和等号
                                if option_text_new not in option_new:
                                    option_new.append(option_text_new)
                                index = option_new.index(option_text_new)
                                option_dict[option_text.split('=')[0].strip()] = index
                        option.append(str(option_dict))
                                    
            for index, value in enumerate(option_new):
                option_new[index] = str(index) + '=' + value

            row['OPTION_new'] = ', '.join(option_new)  # 將OPTION_new設定為當前cluster的唯一選項
            row['OPTION_dict'] = ', '.join(option)  # 設置OPTION_dict的值為OPTION_new
            row['OPTION_pre'] = ', '.join(option_pre)

        # 更新answer_cluster文件
        with open(csv_file_path, 'w', encoding='utf-8', newline='') as csv_file:
            fieldnames = ['ANSWER_new', 'OPTION_dict', 'OPTION_new', 'OPTION_pre', 'ANSWER_pre', 'YEAR', 'NUMBER']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)

        print(f'{filename} 已更新。')

variable_map_type_7.csv 已更新。


查看Answer Organizing輸出結果

In [24]:
df = pd.json_normalize(data)
df

,ANSWER_new,OPTION_dict,OPTION_new,OPTION_pre,ANSWER_pre,YEAR,NUMBER
0,1,,,,"F01A, G01F, labb, LABD, f01a","CVIII2014, CIII2004, CV2008, CII2002, CX2018","f01a, G01, f01a, g01, f01a"
1,2,,,,"F01B, labb, f01b","CVIII2014, CV2008, CX2018","f01b, f01b, f01b"
2,15,,,,"F02A01F, f02b01f","CVIII2014, CX2018","f02a01, f02b01"
3,16,,,,F02A02F,CVIII2014,f02a02
4,17,,,,"F02A03F, f02b03f","CVIII2014, CX2018","f02a03, f02b03"
5,18,,,,"F02A04F, f02b05f","CVIII2014, CX2018","f02a04, f02b05"
6,19,,,,F02A05F,CVIII2014,f02a05
7,20,,,,F02A06F,CVIII2014,f02a06
8,21,,,,F02A07F,CVIII2014,f02a07
9,22,,,,"F02A08F, f02b02f","CVIII2014, CX2018","f02a08, f02b02"


In [29]:
#調查結果大表_part1 -> 會跑很久因為調查人數很多
t = '7'

# 读取原始CSV文件
file_path = f'./data/five_years/five_years_by_type_v2/type_{t}.csv'
data = pd.read_csv(file_path)

# 选择所需的列
selected_columns = ['YEAR', 'cluster', 'QUESTION', 'NUMBER']
data = data[selected_columns]
data.rename(columns={'cluster': 'CLUSTER'}, inplace=True)

# 保存为新的CSV文件
output_path = f'./data/five_years/five_years_survey_data_v2/type_{t}_survey_data.csv'
os.makedirs('./data/five_years/five_years_survey_data_v2', exist_ok=True)
data.to_csv(output_path, index=False)

# 读取已经创建的type_7_survey_data.csv
survey_data_path = f'./data/five_years/five_years_survey_data_v2/type_{t}_survey_data.csv'
survey_data = pd.read_csv(survey_data_path)

# 按'YEAR'列排序
survey_data = survey_data.sort_values(by='YEAR')

# 创建一个新的DataFrame来存储结果
result_data = pd.DataFrame(columns=['ID', 'YEAR', 'CLUSTER', 'QUESTION', 'NUMBER'])

# 初始化上一个年份的变量
last_year = None

# 迭代处理数据
for index, row in survey_data.iterrows():
    current_year = row['YEAR']
    
    # 如果年份变了，打开新的_survey.csv文件
    if current_year != last_year:
        # 关闭上一个_survey.csv文件（如果有的话）
        if last_year is not None:
            survey_file.close()
        
        # 打开新的_survey.csv文件 --這邊是在將SURVEY的回答做MAPPING
        survey_file_path = f'./data/five_years/{current_year}_survey.csv'
        survey_file = open(survey_file_path, 'r')

        # 获取新文件的标题行，并创建title_mapping
        survey_file.seek(0)
        header = next(survey_file)  # 获取标题行
        title_mapping = {title.lower(): idx for idx, title in enumerate(header.strip().split(','))}
    
    # 计算受试者数量
    survey_file.seek(0)
    next(survey_file)  # 跳过标题行
    num_subjects = sum(1 for line in survey_file)
    survey_file.seek(0)  # 重新定位到文件开头
    next(survey_file)
    print('受試者個數:', num_subjects)
    
    # 读取受试者数据并添加到结果中
    for i in range(num_subjects):
        subject_data = next(survey_file).strip().split(',')
        # 提取'data'的值，根据您的描述，'data'是subject_data中，row为subject_data[0]，column为row['NUMBER']的那个值
        #try:
        data_value = subject_data[title_mapping[row['NUMBER'].lower()]]
            # 添加新的行，包括'data'列
        result_data = pd.concat([result_data, pd.DataFrame([{'ID': subject_data[0], 'YEAR': current_year, 'CLUSTER': row['CLUSTER'], 'QUESTION': row['QUESTION'], 'NUMBER': row['NUMBER'], 'ANSWER': data_value}])], ignore_index=True)
        #except:
            #print(current_year, row['NUMBER'])
            #print(title_mapping)
    
    # 更新上一个年份的变量
    last_year = current_year

# 关闭最后一个_survey.csv文件
if last_year is not None:
    survey_file.close()
    
sorted_data = result_data.sort_values(by=['ID', 'CLUSTER', 'YEAR', 'NUMBER'])
sorted_data.reset_index(drop=True, inplace=True)

# 将结果保存到新的CSV文件
result_path = f'./data/five_years/five_years_survey_data_v2/type_{t}_survey_data_processed.csv'
sorted_data.to_csv(result_path, index=False)


受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1382
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 1093
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 809
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 650
受試者個數: 522
受試者個數: 522
受試者個數: 522
受試者個數: 522
受試者個數: 522
受試者個數: 522
受試者個數: 522
受試者個數: 522
受試者

In [30]:
# 產出PROCESS2 (輸出答案dataframe，row為受訪人在特定某年的回答，column為cluster)
t = '7'
 
# 读取原始CSV文件
file_path = f'./data/five_years/five_years_survey_data_v2/type_{t}_survey_data_processed.csv'
data = pd.read_csv(file_path)
 
# 使用pivot_table方法重排数据，使用aggfunc来决定如何处理重复的值，这里使用list函数将重复值存储为列表
pivot_data = pd.pivot_table(data, index=['ID', 'YEAR'], columns='CLUSTER', values='ANSWER', aggfunc=list)

# 重置索引，将 'ID' 和 'YEAR' 作为列
pivot_data.reset_index(inplace=True)
 
# 重新命名列名
pivot_data.columns.name = None
sorted_data = pivot_data.sort_values(by=['ID','YEAR'])
sorted_data.reset_index(drop=True, inplace=True)
 
# 保存到新的CSV文件
output_path = f'./data/five_years/five_years_survey_data_v2/type_{t}_survey_data_processed2.csv'
sorted_data.to_csv(output_path, index=False)

## **Output Survey Outcome**

In [31]:
t = '7'
# 读入type_1_survey_data_processed2.csv并保留第一和第二列以及header
df = pd.read_csv(f'./data/five_years/five_years_survey_data_v2/type_{t}_survey_data_processed2.csv')

# 读入variable_map_type_7.csv并转成df_map
df_map = pd.read_csv(f'./data/five_years/five_years_by_type_v2/variable_map_type_{t}.csv')

# 遍历df中的每一row
for i in range(0, len(df)):
    option_list = []

    # 遍历df的columns（除了第一、第二个以外的header）
    for column in df.columns[2:]:
        # 获取key值
        key = column
        
        # 查找对应的df_map的row
        map_row = df_map[df_map['ANSWER_new'] == key]
        # 如果找到对应的row
        if not map_row.empty:
            # 取出option字典裡的字
            option_str = map_row['OPTION_new'].iloc[0]
            # 問題: 11,19沒有轉成value、nan會轉成跳答
            # 手动解析字符串为字典
            option_dict = {opt.split('=')[0]: opt.split('=')[1] for opt in option_str.split(', ')}
            # 获取列中的值，使用 split(', ') 分割，处理 NaN 值
            values_str = df.iloc[i][column]
            values_list = []
            integer_values = []
            if pd.notna(values_str):
                values_list = ast.literal_eval(values_str)
                # 将字符串值转换为整数，如果无法转换则保留原始值
            
                for value in values_list:
                    try:
                        integer_values.append(int(value))
                    except ValueError:
                        # 处理无法转换为整数的情况，保留原始值
                        integer_values.append(value)
                values_list = integer_values
            print(integer_values)
            # 映射结果
            mapped_values = [option_dict.get(str(val), str(val)) for val in integer_values]
            print(mapped_values)
            # 替换df中的原始值
            df.at[i, column] = ','.join(mapped_values)

df.to_csv(f'./data/five_years/five_years_survey_data_v2/result_type_{t}.csv', mode='w', header=True, index=False)


In [32]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 125)
df[:20]

,ID,YEAR,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,10021.0,CIII2004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0.0],NaN,[0.0],[0.0],[0.0],NaN,[0.0],[0.0],[0.0],[0.0],[1.0],[17.0],[11.0],[19.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0.0],NaN,NaN,NaN,NaN
1,10021.0,CV2008,NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[1.0],[27.0],[20.0],[11.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10022.0,CV2008,NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[1.0],[27.0],[19.0],[50.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10023.0,CV2008,NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[1.0],[27.0],[20.0],[27.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10023.0,CVIII2014,NaN,[0.0],[0.0],[0.0],[0.0],NaN,NaN,[0.0],[0.0],[0.0],NaN,NaN,NaN,NaN,NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[170.0],[62.0]
5,10041.0,CII2002,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0.0],NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0.0],NaN,NaN,NaN,NaN
6,10041.0,CIII2004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0.0],NaN,[0.0],[0.0],[0.0],NaN,[0.0],[0.0],[0.0],[0.0],[1.0],[20.0],[9.0],[10.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0.0],NaN,NaN,NaN,NaN
7,10042.0,CIII2004,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[0.0],NaN,[0.0],[0.0],[0.0],NaN,[0.0],[0.0],[0.0],[0.0],[1.0],[20.0],[10.0],[35.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0.0],NaN,NaN,NaN,NaN
8,10042.0,CV2008,NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[1.0],[26.0],[13.0],[15.0],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10052.0,CVIII2014,NaN,[0.0],[0.0],[0.0],[0.0],NaN,NaN,[0.0],[0.0],[0.0],NaN,NaN,NaN,NaN,NaN,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[156.0],[43.0]
